In [1]:
import os
import warnings
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import HumanMessage
import os
from datetime import datetime
from IPython.display import Markdown, clear_output
import uuid
import hashlib
import random
import string

# Suppress TensorFlow warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
warnings.filterwarnings("ignore")

# Load environment variables
load_dotenv()

def get_api_key(key_name="OPENROUTER_API_KEY"):
    """
    Get API key from environment variables
    """
    api_key = os.getenv(key_name)
    if not api_key:
        raise ValueError(f"Invalid API key: {key_name} not found in environment variables")
    return api_key

def initialize_llm(model_name="meta-llama/llama-3.1-8b-instruct",
                  temperature=0.4,
                  use_streaming=True):
    """
    Initialize LLM
    """
    api_key = get_api_key()
    callbacks = [StreamingStdOutCallbackHandler()]
    llm = ChatOpenAI(
        model_name=model_name,
        temperature=temperature,
        streaming=use_streaming,
        callbacks=callbacks,
        openai_api_key=api_key,
        openai_api_base="https://openrouter.ai/api/v1"
    )
    return llm

llm = initialize_llm()

In [2]:
def save_file(data, filename, uniqueness_method="uuid"):

    folder_name = "Hyde Scores/docs"  # Folder to store output files
    os.makedirs(folder_name, exist_ok=True)  # Creates the folder if it doesn't exist
    
    # Generate unique identifier based on selected method
    if uniqueness_method == "uuid":
        # Generate a UUID (universally unique identifier)
        unique_id = str(uuid.uuid4())[:8]
        
    elif uniqueness_method == "random_string":
        # Generate a random string of letters and digits
        chars = string.ascii_letters + string.digits
        unique_id = ''.join(random.choice(chars) for _ in range(8))
        
    elif uniqueness_method == "hash":
        # Create a hash based on content and current time
        content_hash = hashlib.md5((data + str(datetime.now())).encode()).hexdigest()[:8]
        unique_id = content_hash
        
    elif uniqueness_method == "counter":
        # Use an incrementing counter for files with the same base name
        counter = 1
        while True:
            test_filename = f"{filename}_{counter}.md"
            test_path = os.path.join(folder_name, test_filename)
            if not os.path.exists(test_path):
                unique_id = str(counter)
                break
            counter += 1
            
    elif uniqueness_method == "datetime":
        # Original datetime method
        unique_id = datetime.now().strftime("%Y%m%d%H%M%S")
        
    else:
        # Default to UUID if an invalid method is specified
        unique_id = str(uuid.uuid4())[:8]
    
    # Create the final filename with the unique identifier
    final_filename = f"{filename}_{unique_id}.md"
    file_path = os.path.join(folder_name, final_filename)
    
    # Save the data to the file in the specified path
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(data)
        print(f"File '{file_path}' created successfully.")
    
    # Return the full path of the saved file
    return file_path

def show_md_file(file_path):
    with open(file_path, "r", encoding = "utf-8") as file:
        content = file.read()

    display(Markdown(content))

In [3]:
import os
import google.generativeai as genai
from pinecone import Pinecone

# Connect to Pinecone
def connect_to_pinecone(api_key = os.getenv("PINECONE_API_KEY")):
    """
    Connect to Pinecone with your API key
    """
    # Initialize the Pinecone client with the new API
    pc = Pinecone(api_key=api_key)
    print("Connected to Pinecone")
    return pc

In [4]:
# Create Pinecone index with 768 dimensions to match existing index
def create_pinecone_index(dimension=768, index_name="hyde-index", metric="cosine"):
    """
    Create or connect to a Pinecone index
    """
    pc = connect_to_pinecone()
    
    # Check if index already exists
    if index_name not in pc.list_indexes().names():
        # Create index with the new API - using gcp-starter which is free tier compatible
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric=metric,
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            )
        )
        print(f"Created new index: {index_name}")
    else:
        print(f"Using existing index: {index_name}")
    
    # Return the index
    return pc.Index(index_name)

In [5]:
# Get embeddings that match the 768 dimension requirement
def get_embedding(text, model_name="models/embedding-001"):
    """
    Get 768-dimensional embeddings using Google's text-embedding-gecko model
    
    This model produces 768-dimensional embeddings, which match the Pinecone index
    """
    result = genai.embed_content(
        model=model_name,
        content=text,
        task_type="RETRIEVAL_DOCUMENT"
    )
    
    return result["embedding"]

In [6]:
# Insert documents to Pinecone
def insert_documents_to_pinecone(index, documents, namespace=""):
    """
    Insert documents with their embeddings to Pinecone
    """
    # Generate embeddings that match the dimension of the index (768)
    embeddings = []
    for doc in documents:
        embedding = get_embedding(doc)
        embeddings.append(embedding)
    
    # Prepare data for upsert
    vectors = []
    for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
        vectors.append({
            "id": f"doc_{i}",
            "values": embedding,
            "metadata": {"text": doc}
        })
    
    # Upsert vectors in batches
    batch_size = 100
    for i in range(0, len(vectors), batch_size):
        batch = vectors[i:i+batch_size]
        index.upsert(vectors=batch, namespace=namespace)
    
    print(f"Inserted {len(vectors)} documents into Pinecone")
    return len(vectors)

In [7]:
# Search in Pinecone
def search_pinecone(index, query, top_k=2):
    """
    Search for similar documents in Pinecone
    """
    # Get embedding for the query
    query_embedding = get_embedding(query)
    
    # Query the index
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )
    
    # Format results
    search_results = []
    
    for match in results["matches"]:
        search_results.append((match["metadata"]["text"], match["score"]))
    
    return search_results

In [15]:
# Generate hypothetical document using LLM
def generate_hypothetical_document(llm, query):
    """
    Generate a hypothetical document that would answer the query using an LLM.
    This is the core of the HyDE technique.
    """
    hyde_prompt = f"""Generate a hypothetical document that answers the question below, focusing on a scientific discovery or innovation. The document should:
    - Interpret ambiguity: If the question is vague or broad, infer the most relevant scientific concept, event, or technology based on its intent.
    - Be precise: Include specific details such as the discovery's key contributors, approximate time period, core mechanism or process, and its significance.
    - Be concise: Use clear, keyword-rich language to focus on essential facts and impacts, avoiding speculation or overly general context.
    - Be structured: Organize the content with a brief introduction to the discovery, a description of its mechanism or process, and its scientific or societal impact.
    - Align with scientific history: Ensure factual accuracy by grounding the content in established scientific milestones, relevant to the question’s scope.

    Question: {query}

    Hypothetical Document:"""
    
    response = llm.invoke([HumanMessage(content=hyde_prompt)])
    hypothetical_document = response.content
    
    print("\n--- Generated Hypothetical Document ---")
    print(hypothetical_document)
    print("--------------------------------------\n")
    
    return hypothetical_document

In [16]:
# improve the search function with HyDE
def hyde_search(index, query, llm, use_hyde=True, top_k=2):
    """
    Search using Hypothetical Document Embeddings (HyDE) technique.
    
    Args:
        index: Pinecone index
        query: Original user query
        llm: Language model for generating hypothetical document
        use_hyde: Whether to use HyDE or traditional search
        top_k: Number of results to return
        
    Returns:
        List of (document, score) tuples
    """
    if use_hyde:
        print(f"Original query: {query}")
        # Generate hypothetical document
        hypothetical_doc = generate_hypothetical_document(llm, query)
        
        # Use hypothetical document for embedding and search
        print("Searching with hypothetical document embedding...")
        return search_pinecone(index, hypothetical_doc, top_k)
    else:
        # Use traditional search with original query
        print("Searching with original query embedding...")
        return search_pinecone(index, query, top_k)


In [17]:
corpus = [
    """1. **Penicillin Discovery**: In 1928, Alexander Fleming discovered penicillin, the first widely used antibiotic, when he noticed that a mold growing on a petri dish in his lab inhibited bacterial growth. 
This accidental observation led to the isolation of penicillin, which transformed medicine by providing an effective treatment for bacterial infections like pneumonia and sepsis, previously often fatal. 
By the 1940s, mass production of penicillin saved countless lives during World War II, marking the beginning of the antibiotic era, though overuse later contributed to antimicrobial resistance.""",

    """2. **General Relativity**: Albert Einstein's theory of general relativity, published in 1915, redefined gravity as the curvature of spacetime caused by mass and energy, overturning Newton's classical model. 
This theory explained phenomena like the precession of Mercury's orbit and predicted effects such as gravitational lensing, later confirmed by observations. 
Its practical applications include the precise functioning of GPS satellites, which account for time dilation caused by Earth's gravitational field, revolutionizing navigation and telecommunications.""",

    """3. **DNA Double Helix**: In 1953, James Watson and Francis Crick, building on Rosalind Franklin's X-ray diffraction data, elucidated the double-helix structure of DNA at Cambridge University. 
This discovery revealed how genetic information is stored and replicated, laying the groundwork for molecular biology. 
It enabled advancements like the Human Genome Project, genetic engineering, and personalized medicine, where treatments are tailored to an individual's genetic profile, transforming healthcare and biotechnology.""",

    """4. **Steam Engine**: James Watt's improvements to the steam engine in the 1760s and 1770s, particularly the addition of a separate condenser, vastly increased efficiency, making it a cornerstone of the Industrial Revolution. 
By powering machinery, locomotives, and ships, the steam engine drove mass production and global trade, reshaping economies and urban landscapes. 
Its legacy persists in modern energy systems, though it also contributed to early environmental challenges like coal-based pollution.""",

    """5. **Quantum Computing**: Emerging in the 1980s through work by physicists like Richard Feynman and David Deutsch, quantum computing uses quantum mechanics principles, such as superposition and entanglement, to process information exponentially faster than classical computers for specific tasks. 
Unlike binary bits, quantum bits (qubits) can represent multiple states simultaneously, promising breakthroughs in cryptography, drug discovery, and optimization problems. 
Companies like IBM and Google are advancing practical quantum computers, though challenges like error rates remain.""",

    """6. **mRNA Vaccines**: The development of mRNA vaccines, pioneered by researchers like Katalin Karikó and Drew Weissman in the early 2000s, enabled rapid response to the COVID-19 pandemic in 2020. 
These vaccines deliver mRNA that instructs cells to produce a viral protein, triggering an immune response without using live virus. 
Their adaptability and speed of production have revolutionized vaccine development, offering potential applications for diseases like cancer and HIV, though distribution challenges persist in low-resource settings.""",

    """7. **Exoplanet Discovery**: The first confirmed exoplanet, 51 Pegasi b, was discovered in 1995 by Michel Mayor and Didier Queloz using the radial velocity method, which detects stellar wobbles caused by orbiting planets. 
This finding expanded our understanding of planetary systems, revealing diverse worlds, including “hot Jupiters” and potentially habitable exoplanets. 
Over 5,000 exoplanets have since been identified, driving research into astrobiology and the search for extraterrestrial life via telescopes like Kepler and TESS.""",

    """8. **Polymerase Chain Reaction (PCR)**: Invented by Kary Mullis in 1983, the polymerase chain reaction (PCR) is a technique that amplifies small DNA segments, enabling detailed genetic analysis. 
By cycling through temperature changes to replicate DNA, PCR became essential for genetic testing, forensic science, and diagnosing diseases like COVID-19. 
Its simplicity and precision have made it a cornerstone of biotechnology, though it requires careful control to avoid contamination errors.""",

    """9. **World Wide Web**: In 1989, Tim Berners-Lee proposed the World Wide Web at CERN, creating a system to share information via hyperlinked documents over the internet. 
By developing protocols like HTTP and HTML, he enabled global access to interconnected data, transforming communication, education, and commerce. 
The web's open architecture spurred innovations like social media and e-commerce, though it also raised challenges like misinformation and privacy concerns.""",

    """10. **Gravitational Waves**: In 2015, the Laser Interferometer Gravitational-Wave Observatory (LIGO) detected gravitational waves, ripples in spacetime caused by the merger of two black holes, confirming a prediction of Einstein's general relativity. 
This discovery opened a new field of astronomy, allowing scientists to study cosmic events like neutron star collisions, which produce heavy elements like gold. 
LIGO's precision measurements continue to deepen our understanding of the universe's most extreme phenomena.""",

    """11. **Lithium-Ion Batteries**: Developed in the 1980s by John Goodenough, Akira Yoshino, and others, lithium-ion batteries store energy through the movement of lithium ions, offering high efficiency and rechargeability. 
Commercialized by Sony in 1991, they power smartphones, laptops, and electric vehicles, driving the shift to renewable energy. 
Their lightweight design and longevity have transformed technology, though recycling and ethical sourcing of materials like cobalt remain challenges.""",

    """12. **Higgs Boson**: In 2012, the Large Hadron Collider at CERN confirmed the existence of the Higgs boson, a particle proposed by Peter Higgs and others in the 1960s to explain how particles acquire mass via the Higgs field. 
This discovery completed the Standard Model of particle physics, providing insights into the universe's fundamental structure. 
It has spurred research into new physics, though the particle's fleeting nature requires massive accelerators to study.""",

    """13. **Telescope Invention**: In 1608, Hans Lippershey patented the telescope, a device that magnifies distant objects using lenses. 
Galileo Galilei's refinements enabled observations of Jupiter's moons and Saturn's rings, challenging geocentric models and supporting the Copernican revolution. 
Telescopes remain critical for astronomy, with modern versions like the James Webb Space Telescope probing the universe's earliest galaxies, advancing our cosmic knowledge.""",

    """14. **Haber-Bosch Process**: Developed by Fritz Haber and Carl Bosch in the early 20th century, the Haber-Bosch process synthesizes ammonia from nitrogen and hydrogen, enabling large-scale fertilizer production. 
This innovation boosted agricultural yields, supporting global population growth by making farming more efficient. 
However, its energy-intensive nature and environmental impacts, like nitrogen runoff, have prompted research into sustainable alternatives.""",

    """15. **Superconductivity**: Discovered by Heike Kamerlingh Onnes in 1911, superconductivity occurs when certain materials, cooled to extremely low temperatures, conduct electricity with zero resistance. 
This phenomenon enables powerful electromagnets used in MRI machines and high-speed maglev trains. 
Ongoing research into high-temperature superconductors aims to make the technology more practical, potentially revolutionizing energy transmission and storage."""
]


In [18]:
def generate_markdown_report(results, title="HyDE vs Traditional Search"):
    md = f"# {title}\n\n"
    for i, (query, trad, hyde) in enumerate(results, 1):
        md += f"## Query {i}: `{query}`\n\n"

        md += "### 🔍 Traditional Search\n"
        for doc, score in trad:
            md += f"**Score: {score:.4f}**  \n{doc}\n\n"

        md += "### 🤖 HyDE Search\n"
        for doc, score in hyde:
            md += f"**Score: {score:.4f}**  \n{doc}\n\n"

    return md

In [19]:
from pinecone import Pinecone, ServerlessSpec

# Step 1: Connect + setup
connect_to_pinecone()
llm = initialize_llm()
index = create_pinecone_index(dimension=768)
insert_documents_to_pinecone(index, corpus)

# Step 2: Define queries
queries = [
    "What breakthrough changed how we fight infection",
    "How did we learn more about gravity’s role in the universe?",
    "What discovery unlocked the secrets of heredity?",
    "What invention sparked a global industrial transformation?",
    "What technology could redefine computing in the future?"
    ]

all_results = []

for i, query in enumerate(queries, 1):
    print(f"\n========== QUERY {i}: {query} ==========\n")

    print("\n--- Traditional Search ---")
    trad = search_pinecone(index, query)
    for doc, score in trad:
        print(f"Score: {score:.4f} - {doc}")

    print("\n--- HyDE Search ---")
    hyde = hyde_search(index, query, llm, use_hyde=True)
    for doc, score in hyde:
        print(f"Score: {score:.4f} - {doc}")

    all_results.append((query, trad, hyde))

    # Step 3: Final cleanup and report
    clear_output()

markdown = generate_markdown_report(all_results)
file_path = save_file(markdown, filename="hyde_scores", uniqueness_method="datetime")
show_md_file(file_path)

File 'Hyde Scores/docs/hyde_scores_20250519230708.md' created successfully.


# HyDE vs Traditional Search

## Query 1: `What breakthrough changed how we fight infection`

### 🔍 Traditional Search
**Score: 0.7718**  
1. **Penicillin Discovery**: In 1928, Alexander Fleming discovered penicillin, the first widely used antibiotic, when he noticed that a mold growing on a petri dish in his lab inhibited bacterial growth. 
This accidental observation led to the isolation of penicillin, which transformed medicine by providing an effective treatment for bacterial infections like pneumonia and sepsis, previously often fatal. 
By the 1940s, mass production of penicillin saved countless lives during World War II, marking the beginning of the antibiotic era, though overuse later contributed to antimicrobial resistance.

**Score: 0.7561**  
6. **mRNA Vaccines**: The development of mRNA vaccines, pioneered by researchers like Katalin Karikó and Drew Weissman in the early 2000s, enabled rapid response to the COVID-19 pandemic in 2020. 
These vaccines deliver mRNA that instructs cells to produce a viral protein, triggering an immune response without using live virus. 
Their adaptability and speed of production have revolutionized vaccine development, offering potential applications for diseases like cancer and HIV, though distribution challenges persist in low-resource settings.

### 🤖 HyDE Search
**Score: 0.9021**  
1. **Penicillin Discovery**: In 1928, Alexander Fleming discovered penicillin, the first widely used antibiotic, when he noticed that a mold growing on a petri dish in his lab inhibited bacterial growth. 
This accidental observation led to the isolation of penicillin, which transformed medicine by providing an effective treatment for bacterial infections like pneumonia and sepsis, previously often fatal. 
By the 1940s, mass production of penicillin saved countless lives during World War II, marking the beginning of the antibiotic era, though overuse later contributed to antimicrobial resistance.

**Score: 0.6874**  
3. **DNA Double Helix**: In 1953, James Watson and Francis Crick, building on Rosalind Franklin's X-ray diffraction data, elucidated the double-helix structure of DNA at Cambridge University. 
This discovery revealed how genetic information is stored and replicated, laying the groundwork for molecular biology. 
It enabled advancements like the Human Genome Project, genetic engineering, and personalized medicine, where treatments are tailored to an individual's genetic profile, transforming healthcare and biotechnology.

## Query 2: `How did we learn more about gravity’s role in the universe?`

### 🔍 Traditional Search
**Score: 0.7256**  
2. **General Relativity**: Albert Einstein's theory of general relativity, published in 1915, redefined gravity as the curvature of spacetime caused by mass and energy, overturning Newton's classical model. 
This theory explained phenomena like the precession of Mercury's orbit and predicted effects such as gravitational lensing, later confirmed by observations. 
Its practical applications include the precise functioning of GPS satellites, which account for time dilation caused by Earth's gravitational field, revolutionizing navigation and telecommunications.

**Score: 0.7017**  
10. **Gravitational Waves**: In 2015, the Laser Interferometer Gravitational-Wave Observatory (LIGO) detected gravitational waves, ripples in spacetime caused by the merger of two black holes, confirming a prediction of Einstein's general relativity. 
This discovery opened a new field of astronomy, allowing scientists to study cosmic events like neutron star collisions, which produce heavy elements like gold. 
LIGO's precision measurements continue to deepen our understanding of the universe's most extreme phenomena.

### 🤖 HyDE Search
**Score: 0.8479**  
10. **Gravitational Waves**: In 2015, the Laser Interferometer Gravitational-Wave Observatory (LIGO) detected gravitational waves, ripples in spacetime caused by the merger of two black holes, confirming a prediction of Einstein's general relativity. 
This discovery opened a new field of astronomy, allowing scientists to study cosmic events like neutron star collisions, which produce heavy elements like gold. 
LIGO's precision measurements continue to deepen our understanding of the universe's most extreme phenomena.

**Score: 0.7809**  
2. **General Relativity**: Albert Einstein's theory of general relativity, published in 1915, redefined gravity as the curvature of spacetime caused by mass and energy, overturning Newton's classical model. 
This theory explained phenomena like the precession of Mercury's orbit and predicted effects such as gravitational lensing, later confirmed by observations. 
Its practical applications include the precise functioning of GPS satellites, which account for time dilation caused by Earth's gravitational field, revolutionizing navigation and telecommunications.

## Query 3: `What discovery unlocked the secrets of heredity?`

### 🔍 Traditional Search
**Score: 0.6963**  
3. **DNA Double Helix**: In 1953, James Watson and Francis Crick, building on Rosalind Franklin's X-ray diffraction data, elucidated the double-helix structure of DNA at Cambridge University. 
This discovery revealed how genetic information is stored and replicated, laying the groundwork for molecular biology. 
It enabled advancements like the Human Genome Project, genetic engineering, and personalized medicine, where treatments are tailored to an individual's genetic profile, transforming healthcare and biotechnology.

**Score: 0.6398**  
12. **Higgs Boson**: In 2012, the Large Hadron Collider at CERN confirmed the existence of the Higgs boson, a particle proposed by Peter Higgs and others in the 1960s to explain how particles acquire mass via the Higgs field. 
This discovery completed the Standard Model of particle physics, providing insights into the universe's fundamental structure. 
It has spurred research into new physics, though the particle's fleeting nature requires massive accelerators to study.

### 🤖 HyDE Search
**Score: 0.8818**  
3. **DNA Double Helix**: In 1953, James Watson and Francis Crick, building on Rosalind Franklin's X-ray diffraction data, elucidated the double-helix structure of DNA at Cambridge University. 
This discovery revealed how genetic information is stored and replicated, laying the groundwork for molecular biology. 
It enabled advancements like the Human Genome Project, genetic engineering, and personalized medicine, where treatments are tailored to an individual's genetic profile, transforming healthcare and biotechnology.

**Score: 0.6724**  
4. **Steam Engine**: James Watt's improvements to the steam engine in the 1760s and 1770s, particularly the addition of a separate condenser, vastly increased efficiency, making it a cornerstone of the Industrial Revolution. 
By powering machinery, locomotives, and ships, the steam engine drove mass production and global trade, reshaping economies and urban landscapes. 
Its legacy persists in modern energy systems, though it also contributed to early environmental challenges like coal-based pollution.

## Query 4: `What invention sparked a global industrial transformation?`

### 🔍 Traditional Search
**Score: 0.7215**  
4. **Steam Engine**: James Watt's improvements to the steam engine in the 1760s and 1770s, particularly the addition of a separate condenser, vastly increased efficiency, making it a cornerstone of the Industrial Revolution. 
By powering machinery, locomotives, and ships, the steam engine drove mass production and global trade, reshaping economies and urban landscapes. 
Its legacy persists in modern energy systems, though it also contributed to early environmental challenges like coal-based pollution.

**Score: 0.6955**  
13. **Telescope Invention**: In 1608, Hans Lippershey patented the telescope, a device that magnifies distant objects using lenses. 
Galileo Galilei's refinements enabled observations of Jupiter's moons and Saturn's rings, challenging geocentric models and supporting the Copernican revolution. 
Telescopes remain critical for astronomy, with modern versions like the James Webb Space Telescope probing the universe's earliest galaxies, advancing our cosmic knowledge.

### 🤖 HyDE Search
**Score: 0.8765**  
4. **Steam Engine**: James Watt's improvements to the steam engine in the 1760s and 1770s, particularly the addition of a separate condenser, vastly increased efficiency, making it a cornerstone of the Industrial Revolution. 
By powering machinery, locomotives, and ships, the steam engine drove mass production and global trade, reshaping economies and urban landscapes. 
Its legacy persists in modern energy systems, though it also contributed to early environmental challenges like coal-based pollution.

**Score: 0.6546**  
13. **Telescope Invention**: In 1608, Hans Lippershey patented the telescope, a device that magnifies distant objects using lenses. 
Galileo Galilei's refinements enabled observations of Jupiter's moons and Saturn's rings, challenging geocentric models and supporting the Copernican revolution. 
Telescopes remain critical for astronomy, with modern versions like the James Webb Space Telescope probing the universe's earliest galaxies, advancing our cosmic knowledge.

## Query 5: `What technology could redefine computing in the future?`

### 🔍 Traditional Search
**Score: 0.7792**  
5. **Quantum Computing**: Emerging in the 1980s through work by physicists like Richard Feynman and David Deutsch, quantum computing uses quantum mechanics principles, such as superposition and entanglement, to process information exponentially faster than classical computers for specific tasks. 
Unlike binary bits, quantum bits (qubits) can represent multiple states simultaneously, promising breakthroughs in cryptography, drug discovery, and optimization problems. 
Companies like IBM and Google are advancing practical quantum computers, though challenges like error rates remain.

**Score: 0.6864**  
15. **Superconductivity**: Discovered by Heike Kamerlingh Onnes in 1911, superconductivity occurs when certain materials, cooled to extremely low temperatures, conduct electricity with zero resistance. 
This phenomenon enables powerful electromagnets used in MRI machines and high-speed maglev trains. 
Ongoing research into high-temperature superconductors aims to make the technology more practical, potentially revolutionizing energy transmission and storage.

### 🤖 HyDE Search
**Score: 0.6772**  
5. **Quantum Computing**: Emerging in the 1980s through work by physicists like Richard Feynman and David Deutsch, quantum computing uses quantum mechanics principles, such as superposition and entanglement, to process information exponentially faster than classical computers for specific tasks. 
Unlike binary bits, quantum bits (qubits) can represent multiple states simultaneously, promising breakthroughs in cryptography, drug discovery, and optimization problems. 
Companies like IBM and Google are advancing practical quantum computers, though challenges like error rates remain.

**Score: 0.6280**  
15. **Superconductivity**: Discovered by Heike Kamerlingh Onnes in 1911, superconductivity occurs when certain materials, cooled to extremely low temperatures, conduct electricity with zero resistance. 
This phenomenon enables powerful electromagnets used in MRI machines and high-speed maglev trains. 
Ongoing research into high-temperature superconductors aims to make the technology more practical, potentially revolutionizing energy transmission and storage.



In [13]:
print("\nComparison complete! HyDE generates hypothetical documents that better match the relevant content.")
print("This often leads to improved retrieval performance, especially for complex queries.")


Comparison complete! HyDE generates hypothetical documents that better match the relevant content.
This often leads to improved retrieval performance, especially for complex queries.
